In [25]:
import numpy as np
import json
import pandas as pd
from tqdm import tqdm

Loading the dataset. We have here one dataset that gathered results from 1181 to 2023 for the top-tier Nation :England, Wales, Ireland, Scotland, Italy, France, South Africa, New Zealand, Australia and Argentina. The second dataset present all results for all the World Cup since 1987

In [26]:
rwc_df=pd.read_excel('Data/RugbyWorldCupResults.xlsx',7)
top_tier_df=pd.read_csv('Data/results.csv')
rw_rank=pd.read_csv('Data/world_rugby_rankings.csv')
rw_rank=rw_rank.drop(columns="Unnamed: 0")
rw_rank_model=pd.read_csv('Data/rank_model.csv')
rw_rank_model=rw_rank_model.drop(columns="Unnamed: 0")
top_tier_df["Result"]=[top_tier_df["home_team"][i] if top_tier_df["home_score"][i] else top_tier_df["away_team"][i]  for i in range(len(top_tier_df))]
for i in range(1,len(top_tier_df)):
    if top_tier_df.loc[i,"home_score"]==top_tier_df.loc[i,"away_score"]:
        top_tier_df.loc[i,"Result"]="Draw"

In [27]:
top_tier_df=top_tier_df.drop(columns=["stadium","city","neutral","competition"])
top_tier_df=top_tier_df.drop([i if int(top_tier_df.loc[i,"date"][:4])<1946 else 0 for i in range(1,len(top_tier_df))],axis=0)
top_tier_df["Year"]=[int(top_tier_df["date"][i][:4]) for i in top_tier_df.index]
top_tier_df=top_tier_df.rename(columns={"away_team":"Team 2","home_team":"Team 1","away_score":"Team 2 Score","home_score":"Team 1 Score"})

In [28]:
top_tier_df

,date,Team 1,Team 2,Team 1 Score,Team 2 Score,country,world_cup,Result,Year
479,1946-09-14,New Zealand,Australia,31,8,New Zealand,False,New Zealand,1946
480,1946-09-28,New Zealand,Australia,14,10,New Zealand,False,New Zealand,1946
481,1947-01-01,France,Scotland,8,3,France,False,France,1947
482,1947-01-18,Wales,England,6,9,Wales,False,Wales,1947
483,1947-01-25,Ireland,France,8,12,Ireland,False,Ireland,1947
...,...,...,...,...,...,...,...,...,...
2735,2023-08-12,France,Scotland,30,27,France,False,France,2023
2736,2023-08-19,Wales,South Africa,16,52,Wales,False,Wales,2023
2737,2023-08-19,Ireland,England,29,10,Ireland,False,Ireland,2023
2738,2023-08-25,South Africa,New Zealand,35,7,England,False,South Africa,2023


In [29]:
rw_rank_model

,date,National Team,Rating
0,1871-03-27,Scotland,81.000000
1,1872-02-05,Scotland,79.800000
2,1873-03-03,Scotland,79.840000
3,1874-02-23,Scotland,78.872000
4,1875-02-15,Scotland,78.872000
...,...,...,...
19755,2023-08-05,Italy,73.051397
19756,2023-08-12,Italy,73.051397
19757,2023-08-19,Italy,73.051397
19758,2023-08-25,Italy,73.051397


In [30]:
date=[]
def init_rank(rw_rank_model):
    rank={}
    for i in rw_rank_model.index:
        if rw_rank_model["National Team"][i] not in rank:
            rank[rw_rank_model["National Team"][i]]=[]
    return rank
rank=init_rank(rw_rank_model)
for i in tqdm(range(1978)):
    dating_rank=init_rank(rw_rank_model)
    date.append(rw_rank_model["date"][i])
    dating_rank[rw_rank_model["National Team"][i]].append(rw_rank_model["Rating"][i])
    if rw_rank_model["date"][i]>rw_rank["Date"][0]:
        break     
    for j in range(1977,len(rw_rank_model)):
        if rw_rank_model["date"][i]==rw_rank_model["date"][j]:
                dating_rank[rw_rank_model["National Team"][j]].append(rw_rank_model["Rating"][j])    
    ls=sorted(dating_rank.items(), key= lambda x:x[1],reverse=True)
    for idx,elem in enumerate(ls):
        rank[elem[0]].append(idx+1)


 72%|███████▏  | 1433/1978 [08:21<03:10,  2.86it/s]


In [31]:
np.unique(rank['Ireland'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [32]:
def find_index_rank(date, df):
    for i in df.index:
        if df.loc[i,"Date"]<date:
            if df.loc[i+1, "Date"]>=date:
                return i+1
        elif df.loc[i,"Date"]==date:
            return i

rw_rank_modif=pd.DataFrame()
rw_rank_modif["Date"]=date[:-1]
for key in rank.keys():
    rw_rank_modif[key]=rank[key]

rw_rank_modif

,Date,Scotland,England,Ireland,Wales,Australia,New Zealand,France,South Africa,Argentina,Italy
0,1871-03-27,1,10,2,3,4,5,6,7,8,9
1,1872-02-05,2,1,3,4,5,6,7,8,9,10
2,1873-03-03,2,1,3,4,5,6,7,8,9,10
3,1874-02-23,2,1,3,4,5,6,7,8,9,10
4,1875-02-15,3,1,2,4,5,6,7,8,9,10
...,...,...,...,...,...,...,...,...,...,...,...
1428,2003-08-09,8,1,4,9,3,2,6,5,7,10
1429,2003-08-16,8,1,4,9,3,2,6,5,7,10
1430,2003-08-23,8,1,4,9,3,2,6,5,7,10
1431,2003-08-30,8,2,4,9,3,1,5,6,7,10


In [33]:
rw_rank_modif.loc[find_index_rank(top_tier_df["date"][479],rw_rank_modif),top_tier_df["Team 1"][479]]

2

In [34]:
rw_rank=rw_rank.rename(columns={"United States":"United States of America"})
top_tier_df['Team 1 rank']=[rw_rank.loc[find_index_rank(top_tier_df["date"][i],rw_rank),top_tier_df["Team 1"][i]] 
                                if top_tier_df["date"][i]>rw_rank["Date"][0] else rw_rank_modif.loc[find_index_rank(top_tier_df["date"][i],rw_rank_modif),top_tier_df["Team 1"][i]] for i in top_tier_df.index ]
top_tier_df['Team 2 rank']=[rw_rank.loc[find_index_rank(top_tier_df["date"][i],rw_rank),top_tier_df["Team 2"][i]] 
                                if top_tier_df["date"][i]>rw_rank["Date"][0] else rw_rank_modif.loc[find_index_rank(top_tier_df["date"][i],rw_rank_modif),top_tier_df["Team 2"][i]] for i in top_tier_df.index ]

In [ ]:
rwc_df=rwc_df.rename(columns={"Team 1.1":"Team 2"})
ls_rwc={'1987':'New Zealand','1991':'England', '1995':'South Africa', '1999':'Wales', '2003':'Australia','2007': 'France', '2011':'New Zealand', '2015':'England','2019':'Japan'}
rwc_df["country"]=[ls_rwc[str(rwc_df['Year'][i])] for i in rwc_df.index]
rwc_df["world_cup"]=[True for i in rwc_df.index]

On concatène les 2 dataframes 

In [ ]:
result=pd.concat([top_tier_df,rwc_df], sort=True,ignore_index=True)
result=result.reindex(columns=['Year','Team 1','Team 1 rank','Team 1 Score','Team 2','Team 2 rank','Team 2 Score', 'Result', 'country', 'world_cup'])
result=result.sort_values(by="Year")

In [ ]:
rank_1=[]
for i in result.index:
    if type(result['Team 1 rank'][i])==float:
        try:
            rank_1.append(int(rw_rank.loc[find_index_rank(str(result['Year'][i])+'-09-01',rw_rank),result["Team 1"][i]]))
        except KeyError:
            rank_1.append(35)
    else:
        rank_1.append(result['Team 1 rank'][i])

rank_2=[]
for i in result.index:
    if type(result['Team 2 rank'][i])==float:
        try:
            rank_2.append(rw_rank.loc[find_index_rank(str(result['Year'][i])+'-09-01',rw_rank),result["Team 2"][i]])
        except KeyError:
            rank_2.append(35)
    else:
        rank_2.append(result['Team 2 rank'][i])

result['Team 1 rank']=rank_1
result['Team 2 rank']=rank_2

On supprime ici les matchs en doublon de coupe du monde pour les nations du top tier qui apparraissent donc 2 fois

In [ ]:
result=result.drop_duplicates()
d=[]
for i in result.index:
    for j in result.index:
        if j>i:
            if result["Team 1"][i]==result["Team 2"][j] and result["Team 1"][j]==result["Team 2"][i] and result["Team 1 Score"][i]==result["Team 2 Score"][j] and result["Team 1 Score"][j]==result["Team 2 Score"][i] and result["Year"][i]==result["Year"][j]:
                d.append(j)
result=result.drop(d,axis=0)

In [ ]:
result.to_csv('Data/df_rwc_1946.csv')

FOR RWC 2023

In [ ]:
file=open("Data/RWC2023_ranking.json","r",encoding='UTF8') 
rank=json.loads(file.read())
rank
name=[]
for names in rank['schema']['fields']:
    name.append(names['name'])

cl={}
for i,cla in enumerate(rank['data']):
    cl[f'{i}']=cla

df=pd.DataFrame([cl[f'{i}'] for i in sorted(cl.keys())],columns=name)


In [ ]:
df.to_csv('Data/RWC_2023_ranking.csv')